In [1]:
import time
from IoTPy.core.stream import Stream, StreamArray, run
from IoTPy.agent_types.op import map_element, map_list, map_window
from IoTPy.helper_functions.recent_values import recent_values
from IoTPy.helper_functions.print_stream import print_stream
from IoTPy.concurrency.multicore import get_processes

## Simple example of a multicore program
### Processes and their connecting streams
Look at <b>multicore_specification</b>. The specification states that the program has two processes called p0 and p1. Process p0 has a single input stream <i>x</i> and a single output stream <i>y</i>. Process p1 has a single input stream <i>y</i> and no output streams. Thus, the output <i>y</i> of process p0 is the input of process p1.
<br>
### Streams
Streams are specified by a list of pairs where each pair is a stream name and a stream type. The stream type 'i' identifies integers, 'f' floats and 'd' double. We use stream types to allow processes to share memory in Python 2.7+. In this example, the pair ('x', 'i') says that the program has a stream <i>x</i> of type int.
<br>
### Sources
Process p0 has a <b>source_functions</b> called <i>h</i>. Function <i>h</i> executes in its own thread within process p0; this thread is started when the process is started. Function <i>h</i> has a single argument called <i>proc</i> which is a dummy argument that represents a process. 
<br>
<br>
Function <i>h</i> puts data into stream <i>x</i> when it executes <b>proc.copy_stream()</b>. The thread executing <i>h</i> then sleeps for 0.001 seconds before appending more data to stream <i>x</i>. Finally, the thread signals that the source has terminated appending data to stream <i>x</i> by calling <b>proc.finished_source('x')</b>.
### Process Structure
The source <i>h</i> outputs a stream <i>x</i> which is an input of process p0. The output <i>y</i> of process p0 is an input to process p1.
### Process Computations
The computation of a process is specified by a function with two arguments <i>in_streams</i> and <i>out_streams</i>. The computation carried out by p0 is specified by function <i>f</i> which reads a single input stream, <i>in_streams[0]</i> and write a single output stream, <i>out_streams[0]</i>. This agent makes:
<br>
<br>
<b> y[n] = x[n] + 100 </b>
<br>
<br>
The computation carried out by process p1 is specified by function <i>g</i> which prints <b>2 * y[n]</b> for all n.
<br>
<br>
The source function <i>h</i> sets x[n] to n, and so this multicore process prints:
<br>
<br>
<b> 2 * (n + 100) </b>

In [2]:
def test_simple_multicore():
    # Agent function for process named 'p0'
    def f(in_streams, out_streams):
        map_element(lambda v: v+100, in_streams[0], out_streams[0])

    # Agent function for process named 'p1'
    def g(in_streams, out_streams):
        s = Stream('s')
        map_element(lambda v: v*2, in_streams[0], s)
        print_stream(s, 's')

    # Source thread target for source stream named 'x'.
    def h(proc):
        for i in range(2):
            proc.copy_stream(data=list(range(i*3, (i+1)*3)),
                             stream_name='x')
            time.sleep(0.001)
        proc.finished_source(stream_name='x')

    # The specification
    multicore_specification = [
        # Streams
        [('x', 'i'), ('y', 'i')],
        # Processes
        [
            # Process p0
            {'name': 'p0', 'agent': f, 'inputs':['x'], 'outputs': ['y'], 'sources': ['x'],
             'source_functions':[h]},
            # Process p1
            {'name': 'p1', 'agent': g, 'inputs': ['y']}
        ]
       ]

    # Execute processes (after including your own non IoTPy processes)
    processes = get_processes(multicore_specification)
    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()

test_simple_multicore()

s[0] = 200
s[1] = 202
s[2] = 204
s[3] = 206
s[4] = 208
s[5] = 210


## Example of Three Processes in a Row
This example is the same as the previous one except that it has a third process attached to process p2. The source thread <i>h</i> feeds stream <i>x</i> which is the input to process p0. The output of p0 is stream <i>y</i> which is the input to process p1. The output of p1 is stream <i>z</i> which is the input to process p2.
<br>
<br>
### Streams
[('x', 'i'), ('y', 'i'), ('z', 'i')]
This specifies that this system has three streams called 'x', 'y' and 'z' which contain ints.
### Sources
<b>Source Function</b> <i>h</i>
<br>
This function runs in its own thread. The function puts [0, 1, 2] into the stream called <i>x</i>, then sleeps, and then puts [3, 4, 5] into <i>x</i>. The function then calls <i>finished_source</i> to indicate that it has finished executing and so no further values will be appended to <i>x</i>.
<br>
<br>
This function executes in a thread that runs in process <i>p0</i> because <i>h</i> appears in the specification for <i>p0</i>:
<br>
{'name': 'p0', 'agent': f, 'inputs':['x'], 'outputs': ['y'], 'sources': ['x'], <b>'source_functions':[h]</b>}


In [3]:
def test_three_processes_in_linear_sequence():
    # Agent function for process named 'p0'
    def f(in_streams, out_streams):
        map_element(lambda v: v+100, in_streams[0], out_streams[0])

    # Agent function for process named 'p1'
    def g(in_streams, out_streams):
        map_element(lambda v: v*2, in_streams[0], out_streams[0])

    # Agent function for process named 'p2'
    s = Stream()
    def r(in_streams, out_streams):
        map_element(lambda v: v*3, in_streams[0], s)
        print_stream(s, name='s')

    # Source thread target for source stream named 'x'.
    def h(proc):
        for i in range(2):
            proc.copy_stream(data=list(range(i*3, (i+1)*3)),
                             stream_name='x')
            time.sleep(0.001)
        proc.finished_source(stream_name='x')

    # The specification
    multicore_specification = [
        # Streams
        [('x', 'i'), ('y', 'i'), ('z', 'i')],
        # Processes
        [
            # Process p0
            {'name': 'p0', 'agent': f, 'inputs':['x'], 'outputs': ['y'], 'sources': ['x'],
             'source_functions':[h]},
            # Process p1
            {'name': 'p1', 'agent': g, 'inputs': ['y'], 'outputs': ['z']},
            # Process p2
            {'name': 'p2', 'agent': r, 'inputs': ['z']}
        ]
       ]

    # Execute processes (after including your own non IoTPy processes)
    processes = get_processes(multicore_specification)
    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()

test_three_processes_in_linear_sequence()

s[0] = 600
s[1] = 606
s[2] = 612
s[3] = 618
s[4] = 624
s[5] = 630


### Example of Multicore with NumPy Arrays
For a stream or stream array s of numbers, <b>dtype_float</b>(s) is a StreamArray of type float. The meanings of dtype_int and dtype_double are similar.
<br>
<br>
In this example, the agent functions <i>f</i> and <i>g</i> operate on StreamArrays of floats though the source function <i>h</i> generates a stream of int.
<br>
<br>
Using StreamArray and NumPy operations on arrays are often more efficient than using Stream and regular Python operations on single elements of a stream.

In [4]:
import numpy as np
from IoTPy.helper_functions.type import dtype_float

def test_simple_multicore_with_arrays():
    # Agent function for process named 'p0'
    def f(in_streams, out_streams):
        map_window(np.mean, dtype_float(in_streams[0]), out_streams[0], 
                   window_size=2, step_size=2)

    # Agent function for process named 'p1'
    def g(in_streams, out_streams):
        t = StreamArray('t')
        map_window(max, dtype_float(in_streams[0]), t, 
                   window_size=2, step_size=2)
        print_stream(t, 't')

    # Source thread target for source stream named 'x'.
    def h(proc):
        for i in range(2):
            proc.copy_stream(data=list(range(i*10, (i+1)*10)),
                             stream_name='x')
            time.sleep(0.001)
        proc.finished_source(stream_name='x')

    # The specification
    multicore_specification = [
        # Streams
        [('x', 'i'), ('y', 'f')],
        # Processes
        [
            # Process p0
            {'name': 'p0', 'agent': f, 'inputs':['x'], 'outputs': ['y'], 'sources': ['x'],
             'source_functions':[h]},
            # Process p1
            {'name': 'p1', 'agent': g, 'inputs': ['y']}
        ]
       ]

    # Execute processes (after including your own non IoTPy processes)
    processes = get_processes(multicore_specification)
    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()

test_simple_multicore_with_arrays()

t[0] = 2.5
t[1] = 6.5
t[2] = 10.5
t[3] = 14.5
t[4] = 18.5


In [5]:
from IoTPy.agent_types.merge import zip_map
def test_multicore_with_merge():
    # Agent function for process named 'sine'
    def sine(in_streams, out_streams):
        map_element(np.sin, dtype_float(in_streams[0]), out_streams[0])

    # Agent function for process named 'cosine'
    def cosine(in_streams, out_streams):
        map_element(np.cos, dtype_float(in_streams[0]), out_streams[0])

    # Agent function for process named 'tangent'
    def tangent(in_streams, out_streams):
        map_element(np.tan, dtype_float(in_streams[0]), out_streams[0])

    def coordinate(in_streams, out_streams):
        x, sines, cosines, tangents = in_streams
        def f(lst):
            return lst[0]/lst[1]

        def g(lst):
            error_squared= (lst[0] - lst[1])**2
            return error_squared
    
        ratios = Stream('ratios')
        errors = Stream('errors')
        
        zip_map(f, [sines, cosines], ratios)
        zip_map(g, [ratios, tangents], errors)
        print_stream(errors, 'error')

    # Source thread target for source stream named 'x'.
    def sequence(proc):
        proc.copy_stream(data=np.linspace(0.0, np.pi, 20), stream_name='x')
        proc.finished_source(stream_name='x')

    # The specification
    multicore_specification = [
        # Streams
        [('x', 'f'), ('sines', 'f'), ('cosines', 'f'), ('tangents', 'f')],
        # Processes
        [
            # Process sine
            {'name': 'sine', 'agent': sine, 'inputs':['x'], 'outputs': ['sines']},
            # Process cosine
            {'name': 'cosine', 'agent': cosine, 'inputs':['x'], 'outputs': ['cosines']},
            # Process tangent
            {'name': 'tanget', 'agent': tangent, 'inputs':['x'], 'outputs': ['tangents']},
            # Process coordinator
            {'name': 'coordinator', 'agent': coordinate, 'inputs':['x', 'sines', 'cosines', 'tangents'],
            'sources': ['x'], 'source_functions':[sequence]}
        ]
       ]

    # Execute processes (after including your own non IoTPy processes)
    processes = get_processes(multicore_specification)
    for process in processes: process.start()
    for process in processes: process.join()
        
    for process in processes: process.terminate()

test_multicore_with_merge()

error[0] = 0.0
error[1] = 4.532775296369424e-17
error[2] = 9.352038732536478e-18
error[3] = 1.5999276125932144e-17
error[4] = 3.1239249397473935e-16
error[5] = 5.494881338846501e-15
error[6] = 1.4042744413900362e-15
error[7] = 5.38489818398557e-16
error[8] = 1.1467516413736566e-14
error[9] = 2.7396188640615185e-15
error[10] = 1.3010249835774222e-13
error[11] = 1.2023844398172466e-14
error[12] = 8.553425629591828e-15
error[13] = 5.600000722594124e-15
error[14] = 2.0323707396979525e-15
error[15] = 1.0748316446387377e-15
error[16] = 4.716799997565703e-16
error[17] = 8.88901615354599e-16
error[18] = 3.994877548971383e-17
error[19] = 0.0


In [6]:
import multiprocessing
from IoTPy.agent_types.merge import zip_map, zip_map_sink
def test_multicore_with_returning_results_and_keyword_arguments():
    # Value returned by multiocore
    total = multiprocessing.Array('f', 1)
    num = multiprocessing.Array('i', 1)

    # Agent function for process named 'sine'
    def sine(in_streams, out_streams):
        map_element(np.sin, dtype_float(in_streams[0]), out_streams[0])

    # Agent function for process named 'cosine'
    def cosine(in_streams, out_streams):
        map_element(np.cos, dtype_float(in_streams[0]), out_streams[0])

    # Agent function for process named 'tangent'
    def tangent(in_streams, out_streams):
        map_element(np.tan, dtype_float(in_streams[0]), out_streams[0])

    def coordinate(in_streams, out_streams, total, num):
        x, sines, cosines, tangents = in_streams
        def f(lst):
            return lst[0]/lst[1]

        def g(lst, total, num):
            error_squared= (lst[0] - lst[1])**2
            total[0] += error_squared
            num[0] += 1
    
        ratios = Stream('ratios')
        errors = Stream('errors')
        
        zip_map(f, [sines, cosines], ratios)
        #zip_map(g, [ratios, tangents], errors, total=total, num=num)
        zip_map_sink(g, [ratios, tangents], total=total, num=num)
        print_stream(errors)

    # Source thread target for source stream named 'x'.
    def sequence(proc):
        proc.copy_stream(data=np.linspace(0.0, np.pi, 20), stream_name='x')
        proc.finished_source(stream_name='x')

    # The specification
    multicore_specification = [
        # Streams
        [('x', 'f'), ('sines', 'f'), ('cosines', 'f'), ('tangents', 'f')],
        # Processes
        [
            # Process sine
            {'name': 'sine', 'agent': sine, 'inputs':['x'], 'outputs': ['sines']},
            # Process cosine
            {'name': 'cosine', 'agent': cosine, 'inputs':['x'], 'outputs': ['cosines']},
            # Process tangent
            {'name': 'tanget', 'agent': tangent, 'inputs':['x'], 'outputs': ['tangents']},
            # Process coordinator
            {'name': 'coordinator', 'agent': coordinate, 'inputs':['x', 'sines', 'cosines', 'tangents'],
            'sources': ['x'], 'source_functions':[sequence],
            'keyword_args' : {'total' : total, 'num' : num},}
        ]
       ]

    # Execute processes (after including your own non IoTPy processes)
    processes = get_processes(multicore_specification)
    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()

    print ('total error is ', total[0])
    print ('number of samples is ', num[0])
    print ('')

test_multicore_with_returning_results_and_keyword_arguments()

total error is  1.828153513988573e-13
number of samples is  20



In [7]:
import threading
from IoTPy.agent_types.sink import stream_to_queue
def test_multicore_with_queue():
    q = multiprocessing.Queue()

    # Agent function for process named 'p0'
    def f(in_streams, out_streams):
        map_element(lambda v: v+100, in_streams[0], out_streams[0])

    # Agent function for process named 'p1'
    def g(in_streams, out_streams, q):
        s = Stream('s')
        map_element(lambda v: v*2, in_streams[0], s)
        stream_to_queue(s, q)


    # Source thread target for source stream named 'x'.
    def h(proc):
        for i in range(2):
            proc.copy_stream(data=list(range(i*3, (i+1)*3)),
                             stream_name='x')
            time.sleep(0.001)
        proc.finished_source(stream_name='x')


    # Thread target.def publish_data_from_queue(q):
    def get_data_from_output_queue(q):
        while True:
            v = q.get()
            if v == '_finished': break
            else: print ('value ready by thread from q is ', v)
    # Output thread
    my_thread = threading.Thread(target=get_data_from_output_queue, args=(q,))


    # The specification
    multicore_specification = [
        # Streams
        [('x', 'i'), ('y', 'i')],
        # Processes
        [
            # Process p0
            {'name': 'p0', 'agent': f, 'inputs':['x'], 'outputs': ['y'], 'sources': ['x'],
             'source_functions':[h]},
            # Process p1
            {'name': 'p1', 'agent': g, 'inputs': ['y'], 
             'args': [q], 'output_queues': [q], 'threads': [my_thread]}
        ]
       ]

    # Execute processes (after including your own non IoTPy processes)
    processes = get_processes(multicore_specification)
    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()

test_multicore_with_queue()

value ready by thread from q is  200
value ready by thread from q is  202
value ready by thread from q is  204
value ready by thread from q is  206
value ready by thread from q is  208
value ready by thread from q is  210


In [8]:
def test_echo():
    # This is the delay from when the made sound hits a
    # reflecting surface.
    delay = 4
    # This is the attenuation of the reflected wave.
    attenuation = 0.5
    # The results are put in this queue.
    q = multiprocessing.Queue()

    # Agent function for process named 'p0'
    # This agent zips the sound made with the echo
    # to produce the sound heard.
    def f(in_streams, out_streams, delay):
        sound_made, echo = in_streams
        echo.extend([0] * delay)
        # The zip_map output is the sound heard which is
        # the sound heard plus the echo.
        zip_map(sum, [sound_made, echo], out_streams[0])

    # Agent function for process named 'p1'
    # This process puts the sound heard into the output queue
    #  and returns the echo as an output stream.
    def g(in_streams, out_streams, attenuation, q):
        def gg(v):
            # v is the sound heard
            q.put(v)
            # v*attenuation is the echo
            return v*attenuation
        map_element(
            gg, in_streams[0], out_streams[0])

    # Source thread target for source stream named 'sound_made'.
    # In this test sound made is [0, 1,..., 9, 0, 0, ...., 0]
    def h(proc):
        proc.copy_stream(data=list(range(10)), stream_name='sound_made')
        time.sleep(0.1)
        proc.copy_stream(data=([0]*10), stream_name='sound_made')
        proc.finished_source(stream_name='sound_made')
        return

    # Thread that gets data from the output queue
    # This thread is included in 'threads' in the specification.
    # Thread target
    def get_data_from_output_queue(q):
        finished_getting_output = False
        while not finished_getting_output:
            v = q.get()
            if v == '_finished':
                break
            print ('heard sound = spoken + echo: ', v)
    # Thread
    my_thread = threading.Thread(target=get_data_from_output_queue, args=(q,))

    multicore_specification = [
        # Streams
        [('sound_made', 'f'), ('echo', 'f'), ('sound_heard', 'f')],
        # Processes
        [# Process p0
         {'name': 'p0', 'agent': f, 'inputs': ['sound_made', 'echo'], 'outputs': ['sound_heard'],
          'keyword_args' : {'delay' : delay}, 'sources': ['sound_made'], 'source_functions':[h]},
         # Process p1
         {'name': 'p1', 'agent': g, 'inputs': ['sound_heard'], 'outputs': ['echo'],
          'args': [attenuation, q], 'output_queues': [q], 'threads': [my_thread] } ]]

    # Execute processes (after including your own non IoTPy processes)
    processes = get_processes(multicore_specification)
    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()

test_echo()

heard sound = spoken + echo:  0.0
heard sound = spoken + echo:  1.0
heard sound = spoken + echo:  2.0
heard sound = spoken + echo:  3.0
heard sound = spoken + echo:  4.0
heard sound = spoken + echo:  5.5
heard sound = spoken + echo:  7.0
heard sound = spoken + echo:  8.5
heard sound = spoken + echo:  10.0
heard sound = spoken + echo:  11.75
heard sound = spoken + echo:  3.5
heard sound = spoken + echo:  4.25
heard sound = spoken + echo:  5.0
heard sound = spoken + echo:  5.875
heard sound = spoken + echo:  1.75
heard sound = spoken + echo:  2.125
heard sound = spoken + echo:  2.5
heard sound = spoken + echo:  2.9375
heard sound = spoken + echo:  0.875
heard sound = spoken + echo:  1.0625


In [ ]:
from IoTPy.agent_types.sink import sink_element
def test_single_multicore():
    # Agent function for process named 'p0'
    def f(in_streams, out_streams):
        sink_element(lambda v: print ('v is ', v), in_streams[0])

    # Source thread target for source stream named 'x'.
    def h(proc):
        for i in range(2):
            proc.copy_stream(data=list(range(i*2, (i+1)*2)),
                             stream_name='x')
            time.sleep(0.001)
        proc.finished_source(stream_name='x')

    # The specification
    multicore_specification = [
        # Streams
        [('x', 'i')],
        # Processes
        [
            # Process p0
            {'name': 'p0', 'agent': f, 'inputs':['x'], 'sources': ['x'],
             'source_functions':[h]}
       ]
    ]

    # Execute processes (after including your own non IoTPy processes)
    processes = get_processes(multicore_specification)
    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()

test_single_multicore()

v is  0
v is  1
v is  2
v is  3


In [ ]:
from IoTPy.core.stream import _no_value
from IoTPy.agent_types.merge import blend

mp_array = multiprocessing.Array('i', 4)
def test_feedback_1():
    # Agent function for process named 'p0'
    def f(in_streams, out_streams):
        merged_stream = Stream('merged_stream')
        def r(v, state):
            if state < 4: 
                mp_array[state] = v
                return v, state+1
            else: return _no_value, state+1
        blend(r, in_streams, merged_stream, name='blend', state=0)

        map_element(lambda v: v+1, merged_stream, out_streams[0], name='map')
        
    def g(in_streams, out_streams):
        map_element(lambda v: v+10, in_streams[0], out_streams[0], name='u')

    def h(proc):
        proc.copy_stream(data=[0], stream_name='z')
        proc.finished_source(stream_name='z')

    # The specification
    multicore_specification = [
        # Streams
        [('x', 'i'), ('y', 'i'), ('z', 'i')],
        # Processes
        [
            {'name': 'p0', 'agent': f, 'inputs':['x', 'z'], 'outputs': ['y'], 'sources':['z'],
            'source_functions':[h]},
            {'name': 'p1', 'agent': g, 'inputs':['y'], 'outputs': ['x']}
       ]
    ]

    # Execute processes (after including your own non IoTPy processes)
    processes = get_processes(multicore_specification)
    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()
    print('mp_array')
    print(mp_array[:])

test_feedback_1()

mp_array
[0, 11, 22, 33]


In [ ]:
from IoTPy.core.stream import _no_value
from IoTPy.agent_types.merge import blend

mp_array_0 = multiprocessing.Array('i', 3)
mp_array_1 = multiprocessing.Array('i', 3)
def test_feedback_2():
    # Agent function for process named 'p0'
    def f(in_streams, out_streams):
        merged_stream = Stream('merged_stream')
        def r(v, state):
            if state < 3: 
                mp_array_0[state] = v
                return v, state+1
            else: return _no_value, state+1
        blend(r, in_streams, merged_stream, name='blend', state=0)

        map_element(lambda v: v+1, merged_stream, out_streams[0], name='map')
        
    def g(in_streams, out_streams):
        merged_stream = Stream('merged_stream')
        def r(v, state):
            if state < 3: 
                mp_array_1[state] = v
                return v, state+1
            else: return _no_value, state+1
        blend(r, in_streams, merged_stream, name='blend', state=0)
        map_element(lambda v: v+10, in_streams[0], out_streams[0], name='u')

    def h_x(proc):
        proc.copy_stream(data=[0], stream_name='z_x')
        proc.finished_source(stream_name='z_x')

    def h_y(proc):
        proc.copy_stream(data=[0], stream_name='z_y')
        proc.finished_source(stream_name='z_y')

    # The specification
    multicore_specification = [
        # Streams
        [('x', 'i'), ('y', 'i'), ('z_x','i'),  ('z_y','i')],
        # Processes
        [
            {'name': 'p0', 'agent': f, 'inputs':['x', 'z_x'], 'outputs': ['y'], 'sources':['z_x'],
            'source_functions':[h_x]},
            {'name': 'p1', 'agent': g, 'inputs':['y', 'z_y'], 'outputs': ['x'], 'sources':['z_y'],
            'source_functions':[h_y]}
       ]
    ]

    # Execute processes (after including your own non IoTPy processes)
    processes = get_processes(multicore_specification)
    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()
    print('mp_array_0')
    print(mp_array_0[:])
    print('mp_array_1')
    print(mp_array_1[:])

test_feedback_2()

mp_array_0
[0, 11, 22]
mp_array_1
[1, 0, 12]


In [ ]:
import multiprocessing
from IoTPy.agent_types.merge import zip_map, blend
from IoTPy.core.stream import Stream, StreamArray, run
from IoTPy.core.helper_control import _no_value
from IoTPy.agent_types.op import map_element, map_list, map_window
from IoTPy.helper_functions.recent_values import recent_values
from IoTPy.helper_functions.print_stream import print_stream
from IoTPy.concurrency.multicore import get_processes
import time

def test_grid_3():
    N=8
    even = multiprocessing.Array('f', N)
    odd = multiprocessing.Array('f', N)
    even[1] = 1.0
    odd[1] = 1.0
    even[2] = 1.9
    odd[2] = 1.9
    even[N-1] = N-1
    odd[N-1] = N-1
    # Agent function for controller
    def control(in_streams, out_streams):
        all_finished_step = Stream('all processes finished step')
        initiating = in_streams[0]
        def r(v, state):
            if state < 8: 
                return state, state+1
            else:
                return _no_value, state+1
        zip_map(lambda v: 1, in_streams[1:], all_finished_step, name='zip_map')
        t = Stream('t')
        blend(r, [initiating, all_finished_step], out_streams[0], 
              name='start next step', state=0)
        #map_element(lambda v: v, t, out_streams[0])
        
    def f(in_streams, out_streams, index, even, odd):
        def g(v):
            if (0 < index) and (index < N-1):
                if v%2 == 0:
                    odd[index] = (even[index-1] + even[index] + even[index+1])/3.0
                else:
                    even[index] = (odd[index-1] + odd[index] + odd[index+1])/3.0
            return v+1
        map_element(g, in_streams[0], out_streams[0], name='agent'+str(index))

    def h(proc):
        proc.copy_stream(data=[0], stream_name='initiator')
        proc.finished_source(stream_name='initiator')

    # The specification
    multicore_specification = [
        # Streams
        [('initiator', 'i'), ('start_next_step', 'i')] + [
            ('finished_step'+str(index), 'i') for index in list(range(1, N-1))],
        # Processes
            # Controller
            [{'name': 'controller', 'agent': control, 
             'inputs':['initiator'] + [
                 ('finished_step'+str(index)) for index in list(range(1, N-1))],
             'outputs': ['start_next_step'], 'sources':['initiator'], 'source_functions':[h]}] + \
            # Compute grids
            [{'name': 'grid'+str(index), 'agent': f, 'inputs':['start_next_step'], 
             'outputs': ['finished_step'+str(index)], 'args': [index, even, odd]} for index in list(range(1, N-1))]
    ]

    # Execute processes (after including your own non IoTPy processes)
    processes = get_processes(multicore_specification)
    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()

    print ('even')
    print (even[:])
    print ('odd')
    print (odd[:])

test_grid_3()

Exception in thread controller:
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/kmchandy/Documents/IoTPy/IoTPy/core/compute_engine.py", line 217, in target_of_compute_thread
    self.step()
  File "/Users/kmchandy/Documents/IoTPy/IoTPy/core/compute_engine.py", line 249, in step
    a.next()
  File "/Users/kmchandy/Documents/IoTPy/IoTPy/core/agent.py", line 310, in next
    self.name, len(self._out_lists), len(self.out_streams), self._out_lists)
AssertionError: Error in agent named None. The number of output values, 1, returned by a state transition must equal, 0, the number of output streams. The output values are [[]]

